In [22]:
from maskmm.ipstartup import *
import os
import glob
from os.path import join
import random
import torch

import skimage.io
from skimage.io import imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import random

from maskmm.models.maskrcnn import MaskRCNN
from maskmm import learner
from maskmm.utils import visualize

from maskmm.datasets.nuke.config import Config
from maskmm.datasets.nuke.dataset import Dataset

ROOT_DIR = "/home/ubuntu/maskmm"
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "data/models/mask_rcnn_coco.pth")
DATA = join(expanduser("~"), "data", "nuke") 

config = Config()
config.STEPS_PER_EPOCH = 1
config.VALIDATION_STEPS = 1
torch.manual_seed(123)
random.seed(123)
torch.cuda.manual_seed(123)

[root:INFO]:starting (cellevents.py:36, time=14:27)


time: 12.2 ms


In [23]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

[root:INFO]:starting (cellevents.py:36, time=14:27)


time: 3.04 ms


## Dataset

In [24]:
# create validation sample
np.random.seed(0)
pvalid = .2
trainpath = join(DATA, "stage1_train")

df = pd.DataFrame(os.listdir(trainpath), columns=["image"])
df["subset"] = np.random.random(len(df))>pvalid
df.loc[df.subset==True, "subset"] = "train"
df.loc[df.subset==False, "subset"] = "valid"
df.to_pickle(join(DATA, "subset.pkl"))
df.subset.value_counts()

[root:INFO]:starting (cellevents.py:36, time=14:27)


train    505
valid    123
Name: subset, dtype: int64

time: 40.1 ms


In [25]:
dataset_train = Dataset(config)
dataset_train.load_nuke(trainpath, "train")
dataset_train.prepare()

dataset_val = Dataset(config)
dataset_val.load_nuke(trainpath, "valid")
dataset_val.prepare()

[root:INFO]:starting (cellevents.py:36, time=14:27)


time: 109 ms


In [26]:
%%s
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

[root:INFO]:starting (cellevents.py:36, time=14:27)


time: 2.21 ms


## Create Model

In [27]:
model = MaskRCNN(model_dir=MODEL_DIR, config=config)
model.initialize_weights()

# load pretrained except final layers that depend on NUM_CLASSES
params = torch.load(COCO_MODEL_PATH)
params.pop('classifier.linear_class.weight')
params.pop("classifier.linear_bbox.weight")
params.pop("mask.conv5.weight")
params.pop('classifier.linear_class.bias')
params.pop("classifier.linear_bbox.bias")
params.pop("mask.conv5.bias")
model.load_state_dict(params, strict=False)
if config.GPU_COUNT:
    model = model.cuda()

[root:INFO]:starting (cellevents.py:36, time=14:27)


time: 1.97 s


In [28]:
# Create model in training mode
learner = learner.Learner(model, dataset_train, dataset_val)
learner.train(.01, 1, "heads")

[root:INFO]:starting (cellevents.py:36, time=14:27)



Starting at epoch 1. LR=0.01

Checkpoint Path: /home/ubuntu/maskmm/logs/dsb201810%d_1427/mask_rcnn_dsb_{:04d}.pth
Epoch 1/1.


[root:INFO]:tensor([[[[ 104.3000,  104.3000,  109.3000,  ...,  102.3000,
            101.3000,  106.3000],
          [  98.3000,  102.3000,  108.3000,  ...,   96.3000,
            103.3000,  108.3000],
          [ 108.3000,  111.3000,  105.3000,  ...,  104.3000,
            100.3000,   99.3000],
          ...,
          [ 102.3000,  105.3000,  103.3000,  ...,   65.3000,
             60.3000,   53.3000],
          [ 105.3000,  107.3000,  101.3000,  ...,   59.3000,
             55.3000,   51.3000],
          [ 101.3000,  106.3000,  104.3000,  ...,   54.3000,
             52.3000,   51.3000]],

         [[ 111.2000,  111.2000,  116.2000,  ...,  109.2000,
            108.2000,  113.2000],
          [ 105.2000,  109.2000,  115.2000,  ...,  103.2000,
            110.2000,  115.2000],
          [ 115.2000,  118.2000,  112.2000,  ...,  111.2000,
            107.2000,  106.2000],
          ...,
          [ 109.2000,  112.2000,  110.2000,  ...,   72.2000,
             67.2000,   60.2000],
      

AttributeError: module 'torch.nn.functional' has no attribute 'smooth_l1'

time: 13.2 s


## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
# Which weights to start with?
init_with = "last"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=.001, 
            epochs=10, 
            layers='heads')

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=.001,
            epochs=30, 
            layers="all")

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
# model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
# model.keras_model.save_weights(model_path)

# Predict on valid set

In [ ]:
class InferenceConfig(nuke.NukeConfig):
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = 26 #random.choice(dataset_val.image_ids)
print(image_id)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
results = model.detect([original_image], verbose=1)
r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

# Score

In [ ]:
from tqdm import tqdm_notebook as tqdm
def score(image_ids, verbose=False):
    """ return mean average precision across range of IOU thresholds
    NOTE: takes 3 minutes for 10 using P2 on 1024*1023
    """
    scores = []
    for image_id in tqdm(image_ids):
        # y_true mask
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_val, inference_config,
                                   image_id, use_mini_mask=False)
        y_true = [gt_mask[:, :, i] for i in range(gt_mask.shape[2])]

        # y_pred
        results = model.detect([image], verbose=0)
        masks = results[0]["masks"]
        
        masks = nuke.remove_overlaps(masks)
        
        # convert
        y_pred = [masks[:, :, i] for i in range(masks.shape[2])]

        # score
        if len(masks) == 0:
            score = 0
        else:
            score = nuke.score_image(y_true, y_pred, verbose)
        #log((image_id, score))
        scores.append(score)
    return np.mean(scores), np.std(scores)

In [ ]:
score([36], True)

In [ ]:
score(dataset_val.image_ids)

# Test

In [ ]:
dataset_test = nuke.NukeDataset()
dataset_test.load_nuke(datapath, "stage1_test")
dataset_test.prepare()

In [ ]:
# check a random image from test set
image_id = 40 #random.choice(dataset_test.image_ids)
image = dataset_test.load_image(image_id)
results = model.detect([image], verbose=1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

In [ ]:
# prepare output for submission
out = []
for image_id in dataset_test.image_ids:
    source_id = dataset_test.image_info[image_id]["id"]
    image = dataset_test.load_image(image_id)
    results = model.detect([image], verbose=0)
    masks = results[0]["masks"]
    masks = nuke.remove_overlaps(masks)
    rles = [nuke.rle_encoding(masks[:,:,m]) for m in range(masks.shape[2])]
    for rle in rles:
        out.append(dict(ImageId=source_id, EncodedPixels=rle))
df = pd.DataFrame(out)

In [ ]:
len(df.ImageId.unique())

In [ ]:
# na mask after overlaps removed
df=df[["ImageId", "EncodedPixels"]].dropna()
df.to_csv("out.csv", index=False)
pd.read_csv("out.csv").head()

In [ ]:
print(len(df))
df.ImageId.value_counts()